In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

import import_ipynb
import node_linker

### Critical Distance Function

`def approved_distance(node, radius, vertices, sense)`

Check if the safety radius length does not create conflicts with the edge lenght

#### Parameters:
* **node&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;node_linker.Node**
    * The start node of the analysis that points to a Voronoi vertex in a determined cell.
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Updated vertices of Voronoi Diagram with approximation of infinity vertices
* **sense&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;string**
    * Back or forward sense

#### Return:
* **approve&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * True if there is not a problem

In [ ]:
def approved_distance(node, radius, vertices, sense, verbose):
    coordinate2 = vertices[node.item]
    # Primarly for open cells
    if sense == 'forward':
        coordinate1 = vertices[node.pref.item]
        coordinate3 = vertices[node.nref.item]
        coordinate4 = vertices[node.nref.nref.item]
    # For closed cells
    elif sense == 'back':
        coordinate1 = vertices[node.nref.item]
        coordinate3 = vertices[node.pref.item]
        coordinate4 = vertices[node.pref.pref.item]  
    # Use the Intersections Formula and Sharpen Function in order to reach the
    # critical distance
    inst1 = intersect(coordinate1, coordinate2, coordinate3, radius)
    inst2 = intersect(coordinate2, coordinate3, coordinate4, radius)
    critical_point = sharpen(coordinate2, inst1[1], coordinate3, inst2[1])
    approve = not between(coordinate2, inst1[1], critical_point)
    if verbose:
        print('\nChecking distances in',sense,'sense')
        print('Critical point:',critical_point)
        print('First segment: {',coordinate2,',',inst1[1],'}')
        print('Second segment: {',coordinate3,',',inst2[1],'}')
        print('Approve?',approve,'\n')
    return approve

### Point in a Segment Function

`def between(a, b, c)`

Check inclusion of a point in a segment

#### Parameters:
* **a&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * First vertex of the segment.
* **b&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * Second vertex of the segment.
* **c&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of float**
    * Point to check.    
#### Return:
*  **True/False depending on the inclusion of the point inside the line**

In [ ]:
def between(a, b, c):
    crossproduct = (c[1] - a[1])*(b[0] - a[0]) - (c[0] - a[0])*(b[1] - a[1])
    # compare versus epsilon for floating point values, or != 0 if using
    # integers
    if abs(crossproduct) > 0.01:
        return False
    dotproduct = (c[0] - a[0])*(b[0] - a[0]) + (c[1] - a[1])*(b[1] - a[1])
    if dotproduct < 0:
        return False
    squaredlengthba = (b[0] - a[0])*(b[0] - a[0]) + (b[1] - a[1])*(b[1] - a[1])
    if dotproduct > squaredlengthba:
        return False
    return True

### Collision Free Configuration Function

`def collision(positions, radius)`

Test if the initial position of robots is collision free configuration.

#### Parameters:
* **positions&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * Positions of robots 
* **pointers&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of ints, shape `(nridges, 2)`**
    * Indices of the points between which each Voronoi ridge lies (from scipy.spatial.Voronoi.ridge_points)
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram
    
#### Return:
* **crash&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * False if no collision exists

In [73]:
def collision(positions, pointers, radius):
    crash = False
    i = 0
    while i < len(pointers) and not crash:
        if np.linalg.norm(positions[pointers[i][1]]
                          - positions[pointers[i][0]]) < 2*radius:
            crash = True
        i += 1
    return crash

### Palette Function

`def colors(number):`

Create the color palette

#### Parameters:
* **number&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Number of colors to retun
    
#### Return:
* **palette&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of str**
    * A list with the colors name

In [1]:
def colors(number):
    overlap = ['red','tomato','orangered','orange','goldenrod','gold',
               'yellowgreen','chartreuse','lime','green','darkgreen','teal',
               'turquoise','aqua','cyan','blue','navy','indigo','purple',
               'fuchsia','magenta','crimson','maroon','brown','sienna']
    palette = []
    for i in range(number):
        palette.append(overlap[int(i*(len(overlap)/number))])
    return palette

### Intersection Function

`def intersect(vertice1, origin, vertice2, radius):`

Generate the new BVC vertices from the intersection of the Voronoi edges  
moved according the safety radius in different sectors of the cell

#### Parameters:
* **vertice1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme Voronoi vertex.
* **origin&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector intermediate Voronoi vertex.
* **vertice2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme Voronoi vertex.
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * radius to define the BVC from the Voronoi Diagram

#### Return:
* **new_vertice1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme BVC vertex.
* **intersection&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector intermediate BVC vertex.
* **new_vertice2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Coordinates from one of the sector extreme BVC vertex.

In [4]:
def intersect(vertice1, origin, vertice2, radius):
    # Change coordinates
    vertice1 = vertice1 - origin
    vertice2 = vertice2 - origin
    # Unit vectors
    vertice1_unit = vertice1 / np.linalg.norm(vertice1)
    vertice2_unit = vertice2 / np.linalg.norm(vertice2)
    # Get the angle between edges
    p_dot = np.dot(vertice1_unit, vertice2_unit)
    angle = np.arccos(p_dot)
    mid_angle = angle/2
    # Compute and show the intersection of the BVC
    mid_vertex = (vertice1_unit + vertice2_unit) / 2
    mid_vertex_unit = mid_vertex / np.linalg.norm(mid_vertex)
    separation = radius / np.sin(mid_angle)
    intersection = mid_vertex_unit * separation
    # Revert coordinates
    new_vertice1 = vertice1 + intersection + origin
    new_vertice2 = vertice2 + intersection + origin
    intersection = intersection + origin
    return [new_vertice1, intersection, new_vertice2]

### Vertices Sense Function

`def get_sense(vertices, previous_position)`

Get the sense of a vertices set

#### Parameters:
* **vetices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of ndarrays of float**
    * Current positions of the robots.
* **previous_position&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Coordinates of the previous positions of robots
    
#### Return:
* **sense&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * Order sense of vertices 

In [ ]:
def get_sense(vertices, previous_position):
    #print("Vertex 0 type:", type(vertices[0]))
    #print("Vertex 1 type:", type(vertices[1]))
    #print("Vertex 2 type:", type(vertices[2]))
    #print("Robots path type:", type(previous_position),"\n")
    # Check the type of cell
    if len(vertices) > 2:
        midpoint = vertices[1]
        extreme1 = vertices[0] - midpoint
        extreme2 = vertices[2] - midpoint
    else:
        midpoint = previous_position
        extreme1 = vertices[0] - midpoint
        extreme2 = vertices[1] - midpoint
    # Analyze the crossproduct
    crossproduct = extreme1[0]*extreme2[1] - extreme2[0]*extreme1[1]
    if crossproduct > 0:
        sense = 'clockwise'
    elif crossproduct < 0:
        sense = 'counterclockwise'
    return sense

### Closer Edge Points Function

`def perpendicular_dist(target_point, vertices)`

Get the closest point of an edge to the target point

#### Parameters:
* **target_point&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * The target point of a specific robot.
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * Finite BVC vertices.
    
#### Return:
* **edge_points&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * List of closest points on the BVC edges to the target

In [ ]:
def perpendicular_dist(target_point, vertices):
    #Initialize the list to return
    edge_points = []
    # Analize each edge
    for i in range(len(vertices[:-1])):
        # Check if the edge is vertical
        if vertices[i][0] == vertices[i+1][0]:
            [x, y] = [vertices[i][0], target_point[1]]
        else:
            # Get the perpendicular point on the egde sense to the target
            slope = (vertices[i+1][1] - vertices[i][1])/(vertices[i+1][0]
                                                         - vertices[i][0])
            x = (slope * (vertices[i][1] - target_point[1]
                          - slope * vertices[i][0])
                 - target_point[0]) / (-1 - pow(slope,2))
            y = (x - target_point[0])/(-slope) + target_point[1]
            # Check if the point is inside of edge to include in the returning
            # list
        if between(vertices[i], vertices[i+1], [x,y]):
            edge_points.append([x,y])
    return edge_points

### Position Generation

`pos_gen(num, seed)`

Generate a pseudorandom configurations for initial and final positions

#### Parameters:
* **num&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Number of robots
* **seed&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;int**
    * Seed for pseudorandom
    
#### Return:
* **init&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;array of list of floats**
    * Initial coordinates for robots position
* **fin&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;array of list of floats**
    * Final coordinates for robots position
* **distances&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;list of floats**
    * Distances between initial and final positions

In [15]:
def pos_gen(num, seed):
    np.random.seed(seed)
    init = []
    fin = []
    distances = []
    radius = math.sqrt(2*num)
    offset = (1/(0.4*num) * np.random.random_sample((num*4,)) - 1/(0.4*num))
    
    for i in range(0, 4*num, 4):
        convert = (2*math.pi*i/4)/num
        init.append([round(radius*math.cos(convert) + offset[i]  ,2),
                     round(radius*math.sin(convert) + offset[i+1],2)])
        fin.append([ round(radius*math.cos(convert+math.pi) + offset[i+2],2),
                     round(radius*math.sin(convert+math.pi) + offset[i+3],2)])
        
    return {"i"   : np.array(init),
            "f"   : np.array(fin),
            "dist":[np.linalg.norm(np.array(fin[i]) - np.array(init[i]))
                    for i in range(num)]
           }

### Pruner Function

`def pruner(vertices, verbose)`

Remove intersections of the BVC

#### Parameters:
* **vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * BVC vertices
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;boolean**
    * Flag to print important values

#### Return:
* **pruned_vertices&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of float**
    * BVC vertices without intersections

In [ ]:
def pruner(vertices, verbose, mode):
    pruned_vertices = []
    cell = node_linker.DoublyLinkedList()
    for vertex in vertices:
        cell.insert_at_end(vertex)
    runner = cell.start_node
    if verbose:
        print('Vertices in doubly linked list:')
        cell.traverse_list()
    while not(runner is cell.end_node.pref.pref
          or runner is cell.end_node.pref):
        vanguard = runner.nref.nref
        while vanguard is not cell.end_node:
            vertex_0 = runner.item
            vertex_1 = runner.nref.item
            vertex_2 = vanguard.item
            vertex_3 = vanguard.nref.item
            if verbose:
                print("\nChecking: ",vertex_0,"—",vertex_1,"\nand ",
                      vertex_2,"—",vertex_3)
            critical_point = sharpen(vertex_0,vertex_1,vertex_2,vertex_3)
            intersection = between(vertex_0,vertex_1,critical_point)
            if verbose:
                print('Intersection?:',intersection)
            if intersection:
                inst_node = runner.nref
                while inst_node is not vanguard:
                    inst_node = inst_node.nref
                    cell.delete_node(inst_node.pref)
                vanguard.item = critical_point
                if verbose:
                    print('The new value is: ',vanguard.item)
                    print('Connected with: ',vanguard.nref.item)
            vanguard = vanguard.nref
        runner = runner.nref
    runner = cell.start_node
    while runner is not cell.end_node.nref:
        pruned_vertices.append(runner.item)
        runner = runner.nref
    if verbose:
        print("\nCell size: ", cell.lenght_list)
        cell.traverse_list()
    return pruned_vertices

In [ ]:
def save_image(opt, num, i):
    if opt["save_image"]:
        folder = 'Simple'
        if opt["optimize"]:
            folder = 'Opt'
        plt.savefig(opt["path_image"]+folder+'/'+str(num)+'/fig'+str(i)+'.svg')
        print('Image '+str(i)+' saved successfully!')
        plt.close()
    elif opt["plot"]:
        plt.show()

### Sharpen Function

`def sharpen(head, tail)`

Calculate a new vertex when edges smaller than the safety radius appear 

#### Parameters:
* **aux1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Auxiliary coordinate for vertex1 to generate the peak 
* **vertex1&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * One of the edge vertices
* **vertex2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * The other edge vertices
* **aux2&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Auxiliary coordinate for vertex2 to generate the peak
    
#### Return:
* **peak&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * New coordinate that replace the edge

In [ ]:
def sharpen(aux1, vertex1, vertex2, aux2):
    # Special cases: vertical edge
    if aux1[0] == vertex1[0]:
        slope2 = (vertex2[1] - aux2[1])/(vertex2[0] - aux2[0])
        x = aux1[0]
        y = slope2*(x - aux2[0]) + aux2[1]
    elif aux2[0] == vertex2[0]:
        slope1 = (vertex1[1] - aux1[1])/(vertex1[0] - aux1[0])
        x = aux2[0]
        y = slope1*(x - aux1[0]) + aux1[1]
    # Another cases
    else:
        slope1 = (vertex1[1] - aux1[1])/(vertex1[0] - aux1[0])
        slope2 = (vertex2[1] - aux2[1])/(vertex2[0] - aux2[0])
        x = ((vertex2[1] - aux1[1] + slope1*aux1[0] - slope2*vertex2[0]) /
             (slope1 - slope2))
        y = slope1*(x - aux1[0]) + aux1[1]
    peak = np.array([x,y])
    return peak

In [ ]:
def update_pos(bvc, pos, parameters, covered_dist):
    int_paths = bvc["closer"] - pos["i"]
    new_positions = []
    for i in range(parameters["robots"]):
        if np.all(int_paths[i] == 0):
            # Paralyzed robot for deadlock
            if i in bvc["in_deadlock"]: 
                new_positions.append(bvc["closer"][i])
            else: # Reached target
                new_positions.append(pos["f"][i])
        else:
            candidate = (pos["i"][i] + ((int_paths[i]
                                         / np.linalg.norm(int_paths[i]))
                                        * parameters["movement"]))
            # Reaching the final position
            if between(pos["i"][i], candidate, pos["f"][i]):
                new_positions.append(pos["f"][i])
                fin_dist = pos["i"][i] - pos["f"][i]
                covered_dist[i] += np.linalg.norm(fin_dist)
            # Internal Path less than alpha
            elif (np.linalg.norm(int_paths[i]) < parameters["movement"]):
                new_positions.append(bvc["closer"][i])
                covered_dist[i]+=np.linalg.norm(int_paths[i])
            # Normal movement
            else:
                new_positions.append(candidate)
                covered_dist[i] += parameters["movement"]
    pos["i"] = np.array(new_positions)
    return covered_dist, pos